In [ ]:
import gym
import time
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import wandb
from mygame import Game2048Env
from matplotlib import pyplot as plt
%run ModelAndAgent.ipynb

def get_agent():
    
    fileobj = open('myagent.pkl', 'rb')
    agent = pickle.load(fileobj)
    fileobj.close()
    
    if agent is None:
        agent = Agent()
        agent.env = Game2048Env()
        agent.start_agent(env.observation_space.shape, env.action_space.n)
    
    return agent

def train_agent(train_episodes):
    episodes_before_train = 5
    episodes_before_target_update = 50
        
    agent = get_agent()
    
    for episode in range(1,train_episodes+1):
        agent.run_episode()
        
        if (episode%episodes_before_train==0 and agent.step_num>agent.min_rb_size):
            loss = agent.train_step(agent.sample_size()).detach().item()
            agent.losses.append(loss)
            agent.averageRewards.append(np.mean(agent.episode_rewards))
            agent.epsilons.append(agent.epsilon)
            agent.episodeNum.append(agent.episode)
            agent.episode_rewards = []
            
        if(episode%episodes_before_target_update == 0 and agent.step_num>agent.min_rb_size):
            agent.update_target_model()
        
    fileobj = open('myagent.pkl', 'wb')
    pickle.dump(agent, fileobj)
    fileobj.close()

def test_agent(test_episodes):
    
    agent = get_agent()
    
    for episode in range(1, test_episodes+1):
        print('Episode:', episode, '|', 'Reward:', agent.run_episode(test = True))

if __name__=='__main__':
    
    #test_agent(20)
    #train_agent(1000)
    